In [1]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import pandas as pd

In [2]:
labels_path = "34759_final_project_rect/seq_01/labels.txt"
left_folder = "34759_final_project_rect/seq_01/image_02/data/"
right_folder = "34759_final_project_rect/seq_01/image_03/data/"
calib_path = "34759_final_project_rect/calib_cam_to_cam.txt"


In [3]:
# Load YOLO model
model = YOLO("yolo11l.pt")

In [4]:
def compute_disparity(left_img, right_img):
    # Compute the disparity map from a pair of stereo images

    # Convert to grayscale
    gray_left = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # StereoSGBM parameters
    min_disp = 0
    num_disp = 128  # must be a multiple of 16
    block_size = 7

    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=block_size,
        P1=8*3*block_size**2,
        P2=32*3*block_size**2,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        disp12MaxDiff=1,
        preFilterCap=63
    )
    
    disp = stereo.compute(gray_left, gray_right).astype(np.float32) / 16.0
    return disp


In [5]:
def estimate_bottom_center_3d_from_disp(bbox, disp_map, P_left, P_right, min_valid=5):
    # Estimate the 3D coordinates of the bottom-center point of a bounding box using the disparity map

    x1, y1, x2, y2 = bbox
    H, W = disp_map.shape
    u = int((x1 + x2) / 2)
    v = int(y2 - 1)
    h = y2 - y1
    w = x2 - x1
    patch_h = max(3, int(h * 0.15))
    patch_w = max(5, int(w * 0.30))
    x1p = max(0, u - patch_w//2)
    x2p = min(W-1, u + patch_w//2)
    y1p = max(0, v - patch_h//2)
    y2p = min(H-1, v + patch_h//2)
    patch = disp_map[y1p:y2p+1, x1p:x2p+1]
    valid = patch[(patch > 0) & np.isfinite(patch)]
    if len(valid) < min_valid:
        return None
    disparity = float(np.median(valid))
    f = P_left[0,0]
    cx = P_left[0,2]
    cy = P_left[1,2]
    B = abs(P_right[0,3] / f)
    Z = f * B / disparity
    X = (u - cx) * Z / f
    Y = (v - cy) * Z / f
    return (X, Y, Z)


In [6]:
# --- Load calibration ---
def load_projection_matrices(calib_file):
    # Load camera projection matrices from a calibration file
    P = {}
    with open(calib_file, "r") as cal_f:
        for line in cal_f:
            line = line.strip()
            if line.startswith("P_rect_"):
                key, values = line.split(":")
                cam = int(key.split("_")[-1])
                P[cam] = np.array(list(map(float, values.split()))).reshape(3,4)
    return P

P = load_projection_matrices(calib_path)
P_left = P[2]   # P_rect_02
P_right = P[3]  # P_rect_03

# Baseline in meters
f = P_left[0,0]               # focal length in pixels
B = abs(P_right[0,3] / f)     # baseline = Tx / f
print(f"Focal length: {f:.2f} px, Baseline: {B:.3f} m")

np.set_printoptions(precision=15, suppress=False)
print("P_left (02):\n", P_left)
print("P_right (03):\n", P_right)


Focal length: 707.05 px, Baseline: 0.473 m
P_left (02):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02  4.575831e+01]
 [ 0.000000e+00  7.070493e+02  1.805066e+02 -3.454157e-01]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  4.981016e-03]]
P_right (03):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02 -3.341081e+02]
 [ 0.000000e+00  7.070493e+02  1.805066e+02  2.330660e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  3.201153e-03]]


In [7]:
def project_point(P, X, Y, Z):
    # Project a 3D point (X, Y, Z) into 2D image coordinates using projection matrix P
    pt = np.array([X, Y, Z, 1])
    uvw = P @ pt
    u = uvw[0] / uvw[2]
    v = uvw[1] / uvw[2]
    return int(u), int(v)


In [8]:
def IoU(boxA, boxB):
    # Compute the Intersection over Union (IoU) between two bounding boxes
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) > 0 else 0


In [9]:
# Load GT labels
# Load ground truth labels from a file and organize them by frame
labels = {}
with open(labels_path, "r") as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) < 15:
            continue
        frame = int(parts[0])
        obj_id = int(parts[1])
        cls = parts[2]
        xmin, ymin, xmax, ymax = map(float, parts[6:10])
        x, y, z = map(float, parts[13:16])
        if frame not in labels:
            labels[frame] = []
        labels[frame].append({"id": obj_id, "cls": cls, "bbox": (xmin, ymin, xmax, ymax), "xyz": (x, y, z)})

print(f"Labels loaded for {len(labels)} frames")

# List available frames
available_frames = sorted([int(f.split(".")[0]) for f in os.listdir(left_folder) if f.endswith(".png")])

# --- VIDEO LOOP with correct visualization ---
for frame_id in available_frames:  
    # Read stereo images
    left_img = cv2.imread(os.path.join(left_folder, f"{frame_id:06d}.png"))
    right_img = cv2.imread(os.path.join(right_folder, f"{frame_id:06d}.png"))
    if left_img is None or right_img is None:
        continue

    # Compute disparity map
    disp_map = compute_disparity(left_img, right_img)
    img = left_img.copy()
    h, w = img.shape[:2]

    # -------------------------
    # 1) Draw only GT points
    # -------------------------
    if frame_id in labels:
        for obj in labels[frame_id]:
            Xg, Yg, Zg = obj["xyz"]
            ug, vg = project_point(P_left, Xg, Yg, Zg)
            cv2.circle(img, (ug, vg), 4, (0, 255, 0), -1)  # green GT point
            cv2.putText(img, f"GT: x={Xg:.2f},y={Yg:.2f},z={Zg:.2f}", (ug+5, vg-2),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
            print(f"Frame {frame_id} - GT XYZ: {Xg:.2f}, {Yg:.2f}, {Zg:.2f}")

    # -------------------------
    # 2) YOLO detection
    # -------------------------
    results = model(img, verbose=False)[0]
    persons, bikes = [], []

    for box in results.boxes:
        cls = int(box.cls)
        conf = float(box.conf)
        if conf < 0.6:
            continue
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        bbox = (x1, y1, x2, y2)
        if cls == 0:
            persons.append((bbox, conf))
        elif cls == 1:
            bikes.append((bbox, conf))

    # -------------------------
    # 3) Cyclist recognition
    # -------------------------
    cyclists = []
    used_persons, used_bikes = set(), set()
    for i, (pb, pconf) in enumerate(persons):
        for j, (bb, bconf) in enumerate(bikes):
            if IoU(pb, bb) > 0.1:
                x1 = min(pb[0], bb[0])
                y1 = min(pb[1], bb[1])
                x2 = max(pb[2], bb[2])
                y2 = max(pb[3], bb[3])
                cyclists.append(((x1, y1, x2, y2), max(pconf, bconf)))
                used_persons.add(i)
                used_bikes.add(j)

    # -------------------------
    # 4) Draw YOLO Pedestrian and Cyclist in blue with labels+ID and 3D XYZ near bottom point
    # -------------------------
    pedestrian_id = 0
    cyclist_id = 0

    # Pedestrian
    for idx, (pb, conf) in enumerate(persons):
        if idx in used_persons:
            continue
        x1, y1, x2, y2 = pb
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 1)  # blue bbox
        cv2.putText(img, f"Pedestrian:{pedestrian_id}", (x1, y1-8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

        xyz = estimate_bottom_center_3d_from_disp(pb, disp_map, P_left, P_right)
        if xyz:
            Xp, Yp, Zp = xyz
            up, vp = project_point(P_left, Xp, Yp, Zp)
            cv2.circle(img, (up, vp), 4, (0, 0, 255), -1)  # red 3D point
            cv2.putText(img, f"{Xp:.2f},{Yp:.2f},{Zp:.2f}", (up+5, vp-2),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            print(f"Frame {frame_id} - YOLO Pedestrian ID {pedestrian_id} XYZ: {Xp:.2f}, {Yp:.2f}, {Zp:.2f}")
        pedestrian_id += 1

    # Cyclist
    for (cb, conf) in cyclists:
        x1, y1, x2, y2 = cb
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 1)  # blue bbox
        cv2.putText(img, f"Cyc-{cyclist_id}", (x1, y1-8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

        xyz = estimate_bottom_center_3d_from_disp(cb, disp_map, P_left, P_right)
        if xyz:
            Xc, Yc, Zc = xyz
            uc, vc = project_point(P_left, Xc, Yc, Zc)
            cv2.circle(img, (uc, vc), 4, (0, 0, 255), -1)
            cv2.putText(img, f"{Xc:.2f},{Yc:.2f},{Zc:.2f}", (uc+5, vc-2),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            print(f"Frame {frame_id} - YOLO Cyclist ID {cyclist_id} XYZ: {Xc:.2f}, {Yc:.2f}, {Zc:.2f}")
        cyclist_id += 1

    # -------------------------
    # Display video
    # -------------------------
    cv2.imshow("YOLO + Stereo 3D", img)
    if frame_id == 0:
        key = cv2.waitKey(3000)
    else:
        key = cv2.waitKey(50)

    if key == 27:
        break

cv2.destroyAllWindows()
print(f"Frame {frame_id}: width={w} px, height={h} px")




Labels loaded for 145 frames
Frame 0 - GT XYZ: -0.88, 1.37, 6.82
Frame 0 - GT XYZ: -1.33, 1.40, 5.92
Frame 0 - GT XYZ: -3.44, 0.60, 21.98
Frame 0 - GT XYZ: -2.64, 0.41, 26.11
Frame 0 - GT XYZ: -1.91, 0.36, 25.90
Frame 0 - GT XYZ: -1.08, 0.38, 25.06
Frame 0 - GT XYZ: -3.57, 0.10, 29.95
Frame 0 - YOLO Pedestrian ID 0 XYZ: -1.25, 1.35, 5.43
Frame 0 - YOLO Pedestrian ID 1 XYZ: -0.84, 1.30, 6.20
Frame 1 - GT XYZ: -0.76, 1.38, 6.74
Frame 1 - GT XYZ: -1.24, 1.39, 5.88
Frame 1 - GT XYZ: -3.37, 0.62, 21.81
Frame 1 - GT XYZ: -2.66, 0.38, 26.13
Frame 1 - GT XYZ: -1.90, 0.36, 25.74
Frame 1 - GT XYZ: -1.10, 0.37, 24.94
Frame 1 - GT XYZ: -3.64, 0.11, 29.81
Frame 1 - YOLO Pedestrian ID 0 XYZ: -1.23, 1.34, 5.42
Frame 1 - YOLO Pedestrian ID 1 XYZ: -0.76, 1.31, 6.18
Frame 2 - GT XYZ: -0.64, 1.39, 6.67
Frame 2 - GT XYZ: -1.12, 1.38, 5.83
Frame 2 - GT XYZ: -3.30, 0.63, 21.65
Frame 2 - GT XYZ: -2.65, 0.39, 25.91
Frame 2 - GT XYZ: -1.90, 0.36, 25.59
Frame 2 - GT XYZ: -1.08, 0.37, 24.80
Frame 2 - GT XYZ: -3.

In [10]:
# --- 1) Find available frames ---
# List all PNG frames in the left camera folder
available_frames = sorted([int(f.split(".")[0]) for f in os.listdir(left_folder) if f.endswith(".png")])
print(f"Total available frames: {len(available_frames)}")

# --- 2) Load GT labels ---
# Load ground truth Pedestrian labels from file
labels = {}
with open(labels_path, "r") as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) < 15: 
            continue
        frame = int(parts[0])
        cls = parts[2]
        if cls != "Pedestrian": 
            continue
        xmin, ymin, xmax, ymax = map(float, parts[6:10])
        if frame not in labels: 
            labels[frame] = []
        labels[frame].append((xmin, ymin, xmax, ymax))

# --- 3) YOLO inference and save predictions ---
# Run YOLO detection per frame and estimate 3D positions
predictions_per_frame = {}

for frame_id in available_frames:

    # Read stereo images
    left_img = cv2.imread(os.path.join(left_folder, f"{frame_id:06d}.png"))
    right_img = cv2.imread(os.path.join(right_folder, f"{frame_id:06d}.png"))
    if left_img is None or right_img is None:
        continue

    # Compute disparity map
    disp_map = compute_disparity(left_img, right_img)

    # Run YOLO detection
    results = model(left_img, verbose=False)[0]

    preds = []

    for box in results.boxes:
        cls = int(box.cls)
        conf = float(box.conf)
        if conf < 0.6:
            continue

        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

        # Estimate 3D coordinates of bottom-center of bbox
        xyz = estimate_bottom_center_3d_from_disp((x1, y1, x2, y2), disp_map, P_left, P_right)

        preds.append({
            "cls": cls,  
            "bbox": (x1, y1, x2, y2),
            "xyz": xyz  
        })

    predictions_per_frame[frame_id] = preds


Total available frames: 145


In [11]:


# ============================================================
# 1) PARSING KITTI LABELS CORRECTLY
# ============================================================
# Load and parse KITTI ground truth labels, including 3D positions
labels = {}

with open(labels_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 16:
            continue

        frame = int(parts[0])
        obj_id = int(parts[1])
        cls = parts[2]

        xmin, ymin, xmax, ymax = map(float, parts[6:10])
        x, y, z = map(float, parts[13:16])  # GT 3D position

        if frame not in labels:
            labels[frame] = []

        labels[frame].append({
            "id": obj_id,
            "cls": cls,
            "bbox": (xmin, ymin, xmax, ymax),
            "xyz": (x, y, z)
        })

# ============================================================
# 2) MATCHING GT–PREDICTIONS ONLY TRUE POSITIVES
# ============================================================
# Match ground truth and predicted 3D points for the same object class
class_map = {
    0: "Pedestrian",
    1: "Cyclist"
}

matches = []

for frame in available_frames:

    gt_list = labels.get(frame, [])
    pred_list = predictions_per_frame.get(frame, [])

    used_preds = set()

    for gt in gt_list:

        gt_box = gt["bbox"]
        gt_x, gt_y, gt_z = gt["xyz"]
        gt_id = gt["id"]
        gt_cls = gt["cls"]

        best_iou = 0
        best_pred = None
        best_idx = None

        for i, pred in enumerate(pred_list):

            pred_box = pred["bbox"]
            pred_cls = class_map.get(pred["cls"], None)
            pred_xyz = pred["xyz"]

            if pred_cls is None:
                continue

            if i in used_preds:
                continue

            if pred_cls != gt_cls:
                continue

            iou_val = IoU(gt_box, pred_box)

            if iou_val > 0.5 and iou_val > best_iou:
                best_iou = iou_val
                best_pred = pred
                best_idx = i

        if best_pred is not None and best_pred["xyz"] is not None:
            used_preds.add(best_idx)

            matches.append({
                "gt_id": gt_id,
                "class": gt_cls,
                "frame": frame,
                "gt_x": gt_x,
                "gt_y": gt_y,
                "gt_z": gt_z,
                "pred_x": best_pred["xyz"][0],
                "pred_y": best_pred["xyz"][1],
                "pred_z": best_pred["xyz"][2],
            })

# ============================================================
# 3) CALCULATE MAE FOR EACH GT OBJECT
# ============================================================
# Compute mean absolute error (MAE) for X, Y, Z and Euclidean distance per object
df = pd.DataFrame(matches)

df_result = df.groupby(["gt_id", "class"]).apply(lambda g: pd.Series({
    "frames_present": len(g),
    "mae_x": np.mean(np.abs(g["gt_x"] - g["pred_x"])),
    "mae_y": np.mean(np.abs(g["gt_y"] - g["pred_y"])),
    "mae_z": np.mean(np.abs(g["gt_z"] - g["pred_z"])),
    "mae_euclid": np.mean(np.sqrt(
        (g["gt_x"] - g["pred_x"])**2 +
        (g["gt_y"] - g["pred_y"])**2 +
        (g["gt_z"] - g["pred_z"])**2
    ))
})).reset_index()

df_result

# csv_path = "mae_results.csv"  # you can change the path/filename
# df_result.to_csv(csv_path, index=False)


C:\Users\andre\AppData\Local\Temp\ipykernel_13960\472861102.py:102: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_result = df.groupby(["gt_id", "class"]).apply(lambda g: pd.Series({


,gt_id,class,frames_present,mae_x,mae_y,mae_z,mae_euclid
0,0,Pedestrian,39.0,0.210094,0.262040,0.641616,0.766958
1,1,Pedestrian,34.0,0.124759,0.182755,0.348124,0.458312
2,2,Pedestrian,48.0,0.166096,0.149808,0.597758,0.668584
3,3,Pedestrian,47.0,0.180758,0.234158,0.413648,0.588630
4,4,Pedestrian,37.0,0.185461,0.130139,0.643250,0.706717
5,5,Pedestrian,87.0,0.211000,0.136165,1.423400,1.468709
6,6,Pedestrian,102.0,0.145879,0.144079,1.633411,1.675592
7,7,Pedestrian,108.0,0.132605,0.158679,1.396384,1.440737
8,8,Pedestrian,92.0,0.294603,0.109418,1.566272,1.627299


In [12]:
# Evaluate detection metrics per class
# Compute precision, recall, and F1-score for Pedestrian and Cyclist classes
iou_threshold = 0.5

# Define classes to evaluate: (YOLO class_id, GT class_name)
classes = [
    (0, "Pedestrian"),
    (1, "Cyclist")
]

# Dictionaries to store metrics per class
results = {}

for class_id, class_name in classes:
    total_TP = total_FP = total_FN = 0

    for frame_id in predictions_per_frame:
        # Filter predictions and GT for the current class
        pred_boxes = [p["bbox"] for p in predictions_per_frame[frame_id] if p["cls"] == class_id]
        gt_boxes = [g["bbox"] for g in labels.get(frame_id, []) if g["cls"] == class_name]

        matched_gt = set()
        TP = FP = 0

        for pb in pred_boxes:
            matched = False
            for i, gt in enumerate(gt_boxes):
                if i in matched_gt:
                    continue
                if IoU(pb, gt) >= iou_threshold:
                    TP += 1
                    matched_gt.add(i)
                    matched = True
                    break
            if not matched:
                FP += 1

        FN = len(gt_boxes) - len(matched_gt)

        total_TP += TP
        total_FP += FP
        total_FN += FN

    # Final metrics for this class
    precision = total_TP / (total_TP + total_FP) if total_TP + total_FP > 0 else 0
    recall = total_TP / (total_TP + total_FN) if total_TP + total_FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    results[class_name] = {
        "TP": total_TP,
        "FP": total_FP,
        "FN": total_FN,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Print per-class results
for class_name, metrics in results.items():
    print(f"\nClass: {class_name}")
    print(f"TP={metrics['TP']}, FP={metrics['FP']}, FN={metrics['FN']}")
    print(f"Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}, F1={metrics['f1']:.2f}")

# Overall metrics across all classes
total_TP = sum(m["TP"] for m in results.values())
total_FP = sum(m["FP"] for m in results.values())
total_FN = sum(m["FN"] for m in results.values())

overall_precision = total_TP / (total_TP + total_FP) if total_TP + total_FP > 0 else 0
overall_recall = total_TP / (total_TP + total_FN) if total_TP + total_FN > 0 else 0
overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if overall_precision + overall_recall > 0 else 0

print("\nOverall metrics (all classes):")
print(f"TP={total_TP}, FP={total_FP}, FN={total_FN}")
print(f"Precision={overall_precision:.2f}, Recall={overall_recall:.2f}, F1={overall_f1:.2f}")





Class: Pedestrian
TP=612, FP=65, FN=170
Precision=0.90, Recall=0.78, F1=0.84

Class: Cyclist
TP=0, FP=123, FN=101
Precision=0.00, Recall=0.00, F1=0.00

Overall metrics (all classes):
TP=612, FP=188, FN=271
Precision=0.77, Recall=0.69, F1=0.73


In [13]:
IOU_THRESHOLD = 0.5

def compute_average_precision_for_frame(pred_boxes, gt_boxes, iou_threshold):
    # Compute Average Precision (AP) for a single frame
    # AP = TP / (TP + FP + FN)
    # Simple formula suitable for datasets without prediction scores
    if len(gt_boxes) == 0 and len(pred_boxes) == 0:
        return 1.0   # perfect case
    if len(gt_boxes) == 0 and len(pred_boxes) > 0:
        return 0.0   # all false positives

    matched_gt = set()
    TP = FP = 0

    for pb_dict in pred_boxes:
        pb = pb_dict["bbox"]
        match_found = False
        for i, gt_dict in enumerate(gt_boxes):
            gt = gt_dict["bbox"]
            if i in matched_gt:
                continue
            if IoU(pb, gt) >= iou_threshold:
                TP += 1
                matched_gt.add(i)
                match_found = True
                break
        if not match_found:
            FP += 1

    FN = len(gt_boxes) - len(matched_gt)
    denom = TP + FP + FN
    return TP / denom if denom > 0 else 0.0

# ---------------------------------------------------------
# Compute mAP for multiple classes
# ---------------------------------------------------------
# Evaluate mean Average Precision for Pedestrian, Cyclist, and all classes combined
classes = [
    (0, "Pedestrian"),
    (1, "Cyclist"),
    (None, "All")  # all classes together
]

for class_id, class_name in classes:
    AP_values = []

    for frame_id in predictions_per_frame:
        # Filter predictions and GT for the current class
        if class_id is None:
            pred_boxes = predictions_per_frame[frame_id]        # all predictions
            gt_boxes = labels.get(frame_id, [])
        else:
            pred_boxes = [p for p in predictions_per_frame[frame_id] if p["cls"] == class_id]
            gt_boxes = [g for g in labels.get(frame_id, []) if g["cls"] == class_name]

        AP = compute_average_precision_for_frame(pred_boxes, gt_boxes, IOU_THRESHOLD)
        AP_values.append(AP)

    mAP = np.mean(AP_values) if len(AP_values) > 0 else 0.0
    print(f"mAP for {class_name} @ IoU={IOU_THRESHOLD}: {mAP:.4f}")



mAP for Pedestrian @ IoU=0.5: 0.7688
mAP for Cyclist @ IoU=0.5: 0.0000
mAP for All @ IoU=0.5: 0.5128
